In [1]:
import math
import numpy as np
import torch
import gpytorch
import tqdm
import random
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import sys
import sys
import scipy.io
import torch
import numpy as np 
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from directionalvi.utils.metrics import MSE
sys.path.append("../directionalvi")
#sys.path.append("../../data")
sys.path.append("../directionalvi/utils")
from directional_vi import train_gp

mat = scipy.io.loadmat('MtSH.mat')


in Directional Grad Variational Strategy.py


In [2]:
class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [3]:
x = torch.tensor(np.float64(mat['mth_points'])).float()
y = torch.tensor(np.float64(mat['mth_verts'])).float()
y = y - torch.mean(y)
# data parameters
n   = x.shape[0] 
dim = 2
n_test = n - n//2
print("n is: ", n)

# training params
num_inducing = 30
minibatch_size = 500 #how does minibatch size impact convergence rate?

# generate training data
dataset = TensorDataset(x, y) 
#train-test split
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [n - n_test, n_test], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=minibatch_size, shuffle=True)

n is:  14040


In [8]:
perm = torch.randperm(n)
idx = perm[:num_inducing] 
inducing_points = x[idx, :] #choose inducing points randomly

model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

model.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

In [9]:
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data= n - n_test)

In [10]:
num_epochs = 1000
epochs_iter = tqdm.tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data
    minibatch_iter = tqdm.tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        print(x_batch.shape)
        print(y_batch.shape)
        optimizer.zero_grad()
        output = model(x_batch)
        print(output)
        loss = -mll(output, y_batch)
        print(type(loss))
        print(type(y_batch))
        print(loss.shape)
        #print(loss.item())
        epochs_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()

model.eval()
likelihood.eval()

Epoch:   0%|                                                                                  | 0/1000 [00:00<?, ?it/s]

torch.Size([500, 2])
torch.Size([500, 1])
MultivariateNormal(loc: torch.Size([500]))
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([500])


ValueError: only one element tensors can be converted to Python scalars

# Predict

In [ ]:
for param in model.parameters():
  if param.requires_grad:
    print(param.data)

means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
print("MEANS")
means = means[1:]

# compute MSE
test_mse = MSE(test_y,means)
print(f"\nTesting MSE: {test_mse}")

# Plotting

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(train_x[:,0],train_x[:,1],train_y, color='k')
ax.scatter(train_x[:,0],train_x[:,1],means, color='b')
plt.title("f(x,y) variational fit; actual curve is black, variational is blue")
plt.show()